In [1]:
import numpy as np
import pandas as pd

In [2]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

In [3]:
y_train = train_df.target
X_train = train_df.drop(columns=["target"])

In [4]:
import re
def remove_digits(DF):
    for i in np.arange(len(DF)):
        DF[i] = re.sub(r"\d+", "", DF[i], flags=re.UNICODE)

In [5]:
X_train = X_train[['url', 'title']].apply(lambda x:x[0] + " " + x[1], axis=1)
remove_digits(X_train)

In [6]:
test_df.head(10)

,id,url,title
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ..."
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ..."
4,135313,2gis.ru,67
5,135314,ryazan.gorodrabot.ru,"Вакансия Мерчендайзер визитный в Рязани, Групп..."
6,135315,m.fotosklad.ru,Смартфон Apple iPhone XR 64GB Черный A2105 EU ...
7,135316,sim-dealer.ru,Духовой шкаф Siemens HB337GYS0R - Sim-Dealer. ...
8,135317,krasnoyarsk.hh.ru,Вакансия Аналитик Департамента внедрения инфор...
9,135318,vsetop.org,Anomaly Defenders v1.0 – торрент


In [7]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english') + stopwords.words('russian') + ['ru', 'com', 'net'])

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words=stop_words)

In [9]:
X_train_vectorized = vectorizer.fit_transform(X_train)

In [10]:
from sklearn.model_selection import train_test_split
Data_train, Data_test, Ans_train, Ans_test = train_test_split(X_train_vectorized, y_train, test_size=0.25, random_state=1, shuffle=True)
print(Ans_train)

71141     False
4773      False
16465      True
19126      True
36547     False
          ...  
73349     False
109259    False
50057     False
5192      False
128037    False
Name: target, Length: 101481, dtype: bool


In [75]:
print(Data_train)
from sklearn.linear_model import SGDClassifier
linear_model = SGDClassifier(loss='modified_huber', class_weight="balanced", n_iter_no_change=10, verbose=2)
linear_model.fit(Data_train, Ans_train)

  (0, 47175)	1
  (0, 47825)	2
  (0, 56220)	1
  (0, 70143)	1
  (0, 75377)	1
  (0, 106453)	1
  (0, 115287)	1
  (0, 115742)	1
  (0, 117715)	1
  (0, 117977)	1
  (0, 132418)	1
  (0, 137817)	1
  (0, 153886)	1
  (1, 386)	1
  (1, 53222)	1
  (1, 82008)	1
  (1, 84922)	1
  (1, 97317)	1
  (1, 105737)	1
  (1, 129908)	1
  (1, 144726)	1
  (1, 147177)	1
  (1, 149517)	1
  (1, 149641)	1
  (1, 157022)	1
  :	:
  (101477, 25447)	1
  (101477, 28567)	1
  (101477, 37088)	1
  (101477, 72239)	2
  (101477, 74850)	1
  (101478, 76404)	1
  (101478, 128010)	1
  (101478, 138418)	1
  (101478, 177434)	1
  (101479, 3498)	1
  (101479, 3588)	2
  (101479, 140439)	1
  (101479, 159153)	1
  (101479, 164336)	1
  (101479, 184885)	1
  (101480, 39074)	2
  (101480, 85293)	1
  (101480, 97602)	1
  (101480, 113656)	1
  (101480, 129643)	1
  (101480, 132679)	1
  (101480, 137486)	1
  (101480, 160647)	1
  (101480, 160694)	1
  (101480, 164350)	1
-- Epoch 1
Norm: 91.75, NNZs: 23217, Bias: -1.272509, T: 101481, Avg. loss: 1.408338
Total tra

SGDClassifier(alpha=0.0001, average=False, class_weight='balanced',
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='modified_huber',
              max_iter=1000, n_iter_no_change=10, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=2, warm_start=False)

In [74]:
from sklearn.metrics import f1_score
y_pred = linear_model.predict(Data_test)
print(f1_score(y_pred, Ans_test))

0.9769814902705267


In [135]:
X_test = test_df.drop(columns=[])
X_test = X_test[['url', 'title']].apply(lambda x:x[0] + " " + x[1], axis=1)
remove_digits(X_test)
X_test_vectorized = vectorizer.transform(X_test)

In [136]:
test_df["target"] = linear_model.predict(X_test_vectorized).astype(bool)
test_df[["id", "target"]].to_csv("Smirnov03.csv", index=False)